**Introdução**

Nesse código, será demonstrado como realizar algumas tarefas utilizando LLM e Machine Learning, sendo elas:

- Clusterização
- Classificação
- Sentimentalização

**Pacotes**

Importação de pacotes necessários.

In [111]:
#traducao
from googletrans import Translator
#nlp
import spacy
from collections import Counter
nlp = spacy.load("en_core_web_lg")
#tratamento de dados
import pandas as pd
import numpy as np
#LLM
from transformers import T5EncoderModel, T5Tokenizer
import torch
import torch.nn.functional as F
from transformers import T5ForConditionalGeneration, T5Tokenizer
#Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.cluster import KMeans
from sklearn.neural_network import MLPClassifier


**Dados**

Criando dados necessários para análises psoteriores.

In [112]:
#Criando um DataFrame com comentários fictícios sobre crédito e atendimento de banco
dados = {"text": ["Não consegui aprovação para o crédito, mesmo com bom histórico de pagamento.",
"O atendimento online é muito eficiente e rápido.",
"Consegui um ótimo crédito para financiamento imobiliário, taxas interessantes.",
"A espera para ser atendido na agência é muito longa, precisa melhorar.",
"O limite de crédito oferecido não atende às minhas necessidades.",
"O atendimento via telefone deixou a desejar, muito tempo de espera.",
"Aprovação do crédito foi surpreendentemente rápida, estou satisfeito.",
"O aplicativo do banco facilita muito o atendimento, adoro usar.",
"Recebi uma proposta de crédito muito boa, com condições especiais.",
"Tive um problema, mas o atendimento presencial resolveu rapidamente.",
"Meu crédito pessoal foi aprovado com uma taxa de juros baixa, recomendo.",
"O SAC é muito eficiente, tirou todas as minhas dúvidas rapidamente.",
"O processo de solicitação de crédito é muito burocrático e lento.",
"A gentileza dos atendentes sempre me surpreende, muito bom.",
"Fiquei impressionado com a variedade de opções de crédito disponíveis.",
"O atendimento no chat online é muito prático, resolveu meu problema.",
"Consegui aumentar meu limite de crédito facilmente pelo app.",
"O tempo de espera para atendimento no caixa é inaceitável.",
"A equipe de crédito foi muito atenciosa, me ajudou a escolher a melhor opção.",
"O atendimento no call center precisa de mais treinamento, não souberam me ajudar.",
"A análise de crédito foi mais rápida do que eu esperava, muito eficiente.",
"Prefiro o atendimento presencial, sempre sou bem atendido.",
"Meu pedido de crédito foi negado sem muitas explicações.",
"A experiência de atendimento digital é excelente, muito intuitivo.",
"O crédito consignado oferecido tem taxas competitivas, estou considerando.",
"Sempre sou atendido rapidamente quando vou à agência.",
"A linha de crédito para empresas do banco é muito boa, ajudou minha empresa a crescer.",
"O atendimento por videoconferência é uma ótima opção, muito conveniente.",
"Fui surpreendido com uma oferta de crédito pré-aprovado, muito bom.",
"O processo de abertura de conta e atendimento inicial foi muito ágil e sem complicações."],

"label": ["crédito", "atendimento", "crédito", "atendimento", "crédito", "atendimento",
"crédito", "atendimento", "crédito", "atendimento", "crédito", "atendimento","crédito", 
"atendimento", "crédito", "atendimento", "crédito", "atendimento","crédito", "atendimento",
 "crédito", "atendimento", "crédito", "atendimento","crédito", "atendimento", "crédito",
   "atendimento", "crédito", "atendimento"]
}

#Convertendo o dicionário em DataFrame
df_avaliacoes = pd.DataFrame(dados)

**Tradução**

Tradução do português para o inglês, para otimizar performance do LLM.

In [113]:
#criar funcao para traducao
def tradutor(texto):
  translator = Translator() 
  texto_final = translator.translate(texto, dest=str("en"),src=str("pt")).text

  return texto_final

In [114]:
#aplicando traducao
df_avaliacoes['text_en'] = df_avaliacoes['text'].apply(tradutor)
df_avaliacoes.head(5)

,text,label,text_en
0,"Não consegui aprovação para o crédito, mesmo c...",crédito,"I was unable to get approved for credit, even ..."
1,O atendimento online é muito eficiente e rápido.,atendimento,The online service is very efficient and fast.
2,Consegui um ótimo crédito para financiamento i...,crédito,"I got great credit for real estate financing, ..."
3,A espera para ser atendido na agência é muito ...,atendimento,The wait to be seen at the agency is very long...
4,O limite de crédito oferecido não atende às mi...,crédito,The credit limit offered does not meet my needs.


**Pré-processamento**

In [115]:
#lematizar palavras
def lematiza(text):
  doc = nlp(text.lower())
  tokens_alpha = [token.lemma_ for token in doc if token.is_alpha]
  return " ".join(tokens_alpha)

#lematizar aplicando funcao ja definida
df_avaliacoes['text_clean'] = df_avaliacoes['text_en'].map(lambda x : lematiza(x))

#Converter a lista de textos em uma única string para processamento de frequência
all_texts = ' '.join(df_avaliacoes['text_clean'].tolist())
#Processamento de frequência das palavras
word_frequencies = Counter(all_texts.split())
#Calcular a porcentagem de frequência de cada palavra
total_documents = len(df_avaliacoes)
word_percentages = {word: freq / total_documents for word, freq in word_frequencies.items()}
#Filtrar palavras que aparecem mais de 50% das vezes ou menos que 10%
filtered_words = [word for word, percentage in word_percentages.items() if percentage > 0.50 or percentage < 0.10]
# Remover palavras filtradas dos documentos
df_avaliacoes['text_clean'] = df_avaliacoes['text_clean'].apply(lambda texto: ' '.join([word for word in texto.split() if word not in filtered_words]))

In [116]:
#modelos
tokenizer = T5Tokenizer.from_pretrained("t5-small")
encoderT5 = T5EncoderModel.from_pretrained('t5-small')
modelGen = T5ForConditionalGeneration.from_pretrained("t5-small")


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


**Clusterização**

In [125]:
def gerar_embedding(texto):
    inputs = tokenizer(texto, return_tensors="pt", padding=True, truncation=True, max_length=512)
    outputs = encoderT5(**inputs)
    return outputs.last_hidden_state.mean(dim=1).detach().numpy()

df_cluster = df_avaliacoes[['text_clean','label']]
df_cluster['embedding'] = df_avaliacoes['text_clean'].apply(gerar_embedding)

C:\Users\eduar\AppData\Local\Temp\ipykernel_15100\1469992798.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cluster['embedding'] = df_avaliacoes['text_clean'].apply(gerar_embedding)


In [146]:
embeddings = list(df_cluster['embedding'])
embeddings_array = np.array(embeddings).squeeze() 
#Aplicando K-means
kmeans = KMeans(n_clusters=2).fit(embeddings_array)
#Os rótulos dos clusters para cada ponto de dado (comentário)
df_cluster['cluster'] = kmeans.labels_
#resultados
agrupamento = df_cluster.groupby(['label', 'cluster']).size().unstack(fill_value=0)
agrupamento

c:\Users\eduar\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\eduar\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1440: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


cluster,0,1
label,,
atendimento,0,15
crédito,15,0


**Classificação binária**

In [121]:
def get_t5_embeddings(texts):
    embeddings = []
    for text in texts:
        input_ids = tokenizer(text, return_tensors="pt", padding=True)["input_ids"]
        with torch.no_grad():
            outputs = encoderT5(input_ids=input_ids)
        embeddings.append(outputs.last_hidden_state[:, 0, :].squeeze().numpy())
    return embeddings

#Obter embeddings para os textos
df_class = df_avaliacoes[['text_clean','label']]
df_class['embeddings'] = get_t5_embeddings(df_class['text_clean'].tolist())

C:\Users\eduar\AppData\Local\Temp\ipykernel_15100\1748586751.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['embeddings'] = get_t5_embeddings(df['text_clean'].tolist())


In [152]:
# Preparar os dados
X = np.array(df['embeddings'].tolist())
y = df['label'].factorize()[0]  # Convertendo rótulos em valores numéricos

#Dividindo treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

#model
model = MLPClassifier()
model.fit(X_train, y_train)

#avaliacao
predictions = model.predict(X_test)
accuracia = accuracy_score(y_test, predictions)
print(f"Acurácia: {accuracia:.4f}")

Acurácia: 1.0000


**Sentiment**

In [123]:
#https://discuss.huggingface.co/t/t5-classification-using-text2text/504/8

In [153]:
def Sentimentos(text):
  with torch.no_grad():
    #modelo
    enc = tokenizer("sst2 sentence: "+text, return_tensors="pt")
    decoder_input_ids = torch.tensor([tokenizer.pad_token_id]).unsqueeze(0) 
    logits = modelGen(**enc, decoder_input_ids=decoder_input_ids)[0]
    logits = logits.squeeze(1)
    select_logits = logits[:, [1465, 2841]] 
    #prob
    probs_positive = F.softmax(select_logits, dim=1)[:, 0]
    sentimento = lambda x: "Positivo" if x > 0.8 else ("Negativo" if x < 0.2 else "Neutro")
    
  return sentimento(probs_positive.item())

df_sentimentos = df_avaliacoes[['text','text_en']]
df_sentimentos['sentiment'] = df_sentimentos['text_en'].apply(Sentimentos)
df_sentimentos.head(10)

C:\Users\eduar\AppData\Local\Temp\ipykernel_15100\3895147539.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sentimentos['sentiment'] = df_sentimentos['text_en'].apply(Sentimentos)


,text,text_en,sentiment
0,"Não consegui aprovação para o crédito, mesmo c...","I was unable to get approved for credit, even ...",Negativo
1,O atendimento online é muito eficiente e rápido.,The online service is very efficient and fast.,Positivo
2,Consegui um ótimo crédito para financiamento i...,"I got great credit for real estate financing, ...",Positivo
3,A espera para ser atendido na agência é muito ...,The wait to be seen at the agency is very long...,Negativo
4,O limite de crédito oferecido não atende às mi...,The credit limit offered does not meet my needs.,Negativo
5,"O atendimento via telefone deixou a desejar, m...",The telephone service left something to be des...,Negativo
6,Aprovação do crédito foi surpreendentemente rá...,"Credit approval was surprisingly quick, I am s...",Positivo
7,O aplicativo do banco facilita muito o atendim...,"The bank's app makes service a lot easier, I l...",Positivo
8,"Recebi uma proposta de crédito muito boa, com ...","I received a very good credit offer, with spec...",Positivo
9,"Tive um problema, mas o atendimento presencial...","I had a problem, but the in-person service res...",Neutro
